# Imports

In [1]:
import cv2
import math
import numpy as np
import socket
import matplotlib.pyplot as plt
from cv2 import aruco
from scipy.interpolate import interp1d
mapper=interp1d([0,90],[155,255])
mapperd=interp1d([0,2000],[155,255])

# Aruco Markers Initialization

In [2]:
# Aruco Parameters
num_mark = 20 #Number of markers
size_mark = 500 #Size of markers


### Create Aruco markers 
dict_aruco = aruco.Dictionary_get(aruco.DICT_4X4_50)

front_marker_idx = 1
front_marker = aruco.drawMarker(dict_aruco, front_marker_idx, size_mark)
# cv2.imshow("fd ",front_marker)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
rear_marker_idx = 2
rear_marker = aruco.drawMarker(dict_aruco, rear_marker_idx, size_mark)
target1_marker_idx = 3
target2_marker_idx = 4
target3_marker_idx = 5
target1_marker = aruco.drawMarker(dict_aruco, target1_marker_idx, size_mark) 
target2_marker = aruco.drawMarker(dict_aruco, target2_marker_idx, size_mark)
target3_marker = aruco.drawMarker(dict_aruco, target3_marker_idx, size_mark)
# cv2.imshow("fjd ",target_1_marker)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
target_markers = [target1_marker, target2_marker, target3_marker]
target_marker_ids = [target1_marker_idx, target2_marker_idx, target3_marker_idx]

parameters = aruco.DetectorParameters_create()


# Helper Functions

In [3]:
def preprocess_corners(corners, ids):  
    ids = ids.tolist()
    if front_marker_idx in ids:
        front_corner_idx = ids.index(front_marker_idx)
        front_marker_centre = get_marker_centre(corners[front_corner_idx])
    else:
        front_marker_centre = None
                 
    if rear_marker_idx in ids:
        rear_corner_idx = ids.index(rear_marker_idx)
        rear_marker_centre = get_marker_centre(corners[rear_corner_idx])
    else:
        rear_marker_centre = None 
        
    if target1_marker_idx in ids:
        target1_corner_idx = ids.index(target1_marker_idx)
        target1_marker_centre = get_marker_centre(corners[target1_corner_idx])
    else:
        target1_marker_centre = None
  
        
    if target2_marker_idx in ids:
        target2_corner_idx = ids.index(target2_marker_idx)
        target2_marker_centre = get_marker_centre(corners[target2_corner_idx])
    else:
        target2_marker_centre = None
        
    if target3_marker_idx in ids:
        target3_corner_idx = ids.index(target3_marker_idx)
        target3_marker_centre = get_marker_centre(corners[target3_corner_idx])
    else:
        target3_marker_centre = None
        

    return [front_marker_centre, rear_marker_centre, target1_marker_centre, target2_marker_centre, target3_marker_centre]
            
def get_robot_centre(front_marker_centre, rear_marker_centre):
    return np.array([(front_marker_centre[0]+rear_marker_centre[0])/2 , (front_marker_centre[1]+rear_marker_centre[1])/2])


def get_marker_centre(corner):
    cornerUL = corner[0][0]
    cornerUR = corner[0][1]
    cornerBR = corner[0][2]
    cornerBL = corner[0][3]
    return np.array([ (cornerUL[0]+cornerBR[0])/2 , (cornerUL[1]+cornerBR[1])/2 ])

# def get_vehicle_horizontal_orientation(left_wheel_pos, right_wheel_pos):


# def get_robot_heading(front_centre, rear_centre):
#     return front_centre - rear_centre # as np array

### Corrected: robot heading is the direction or angle
def get_robot_heading(front_centre, rear_centre):
    return get_vector_angle(front_centre - rear_centre) # as np array

def get_target_vector(robot_centre, target_centre):
    return target_centre - robot_centre # as np array

def get_vector_angle(vector):
    return math.atan2(vector[0],vector[1])*(180/math.pi)

def get_heading_error(robot_heading, target_angle):
    return (target_angle - robot_heading + 180) % 360 - 180

def get_rotation_direction(heading_error):
    #mode 1 rotates counter clockwise
    #mode -1 rotates clockwise
    if abs(heading_error) <= 90:
        return int(np.sign(heading_error) )
    return - int(np.sign(heading_error))

def get_abs_heading_error(heading_error):
    if abs(heading_error) <= 90:
        return abs(heading_error)
    return abs(180-abs(heading_error)) 

def get_distance_to_target(robot_centre, target_centre):
    return np.linalg.norm(get_target_vector(robot_centre, target_centre))

def get_longitudinal_direction(heading_error):
    if abs(heading_error) <= 90:
        return 0 #mode 0 moves forward
    return 2 #mode 2 moves backwards

# Communication initialization

In [4]:

# Replace with the IP address of the ESP32
UDP_IP = "192.168.43.57"
# UDP_IP = "192.168.164.112"
UDP_PORT = 8080
# Create a UDP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

font                   = cv2.FONT_HERSHEY_SIMPLEX
fontScale              = 1
fontColor              = (0,0,255)
thickness              = 2
lineType               = 2
camera_on = True
if camera_on:
    cap = cv2.VideoCapture(0)

# Main()

In [ ]:
### Load the video
PWM1=0
PWM2=0
mode=0
#to be tuned
kpr=0.05
kpd=3
# if not camera_on:
#     kpd=3.3
# else:
#     kpd=5
kpoff=0.2
while True:
    # Capture a frame
    if camera_on:
        ret, frame = cap.read()
    files = ['testBack.png', 'testClock.png', 'testFor2.png', 'testBack2.png', 'testCounter2.png',
             'testFor3.png', 'testBack.png', 'testCounter.png', 'testFor.png']
    if not camera_on:
        frame = cv2.imread(files[-1])
    
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Original Image', gray)
    cv2.waitKey(1)
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, dict_aruco, parameters=parameters)
    list_ids = np.ravel(ids)
    
    front_marker_centre, rear_marker_centre, target1_marker_centre, \
     target2_marker_centre, target3_marker_centre = preprocess_corners(corners, list_ids)
    #TODO user input
    if  front_marker_centre is None or  rear_marker_centre is None or target1_marker_centre is None:
        message = str(0) + ',' + str(0) + ',' + str(0)
        data = message.encode()
        sock.sendto(data, (UDP_IP, UDP_PORT))
        continue
    target_centre = target1_marker_centre
    robot_centre= get_robot_centre(front_marker_centre, rear_marker_centre)
    robot_heading= get_robot_heading(front_marker_centre, rear_marker_centre)
    target_vector=get_target_vector(robot_centre, target_centre)
    target_angle=get_vector_angle(target_vector)
    heading_error=get_heading_error(robot_heading, target_angle)
    absHeadingError=get_abs_heading_error(heading_error)
    
    # Drawing on frame
    bottomLeftCornerOfText = tuple(robot_centre.astype(int))
    cv2.arrowedLine(frame, tuple(robot_centre.astype(int)), tuple(target_centre.astype(int)), (255,0,0), thickness=3) 
    cv2.arrowedLine(frame, tuple(robot_centre.astype(int)), tuple(front_marker_centre.astype(int)), (255,0,0), thickness=3) 
    cv2.putText(frame,str(round(heading_error))+" + mode: "+str(get_longitudinal_direction(heading_error)),
    bottomLeftCornerOfText,
    font,      
    fontScale, 
    fontColor, 
    thickness, 
    lineType)
    
    cv2.imshow('Original Image', frame)
    cv2.waitKey(1)
    
    
    if (absHeadingError>5):
        #rotate around itslef
        controledAngleError = kpr * absHeadingError
        PWM1 = mapper(controledAngleError)
        PWM2 = PWM1
        mode = get_rotation_direction(heading_error)
        
    else:
        #Move to target
        errord = get_distance_to_target(robot_centre, target_centre)
        if(errord<20):
            avgSpeed = 0
        else:
            print(kpd*errord)
            avgSpeed = mapperd(min(kpd*errord, 2000))
        if(absHeadingError < 2):
            offset = 0
        else:
            offset = kpoff * absHeadingError
        mode = get_longitudinal_direction(heading_error)
        rotationDirection = get_rotation_direction(heading_error)
        print("rotationDirection ",rotationDirection)
        # at mode zero(Forwards) PWM1 Left------PWM2 Right
        #at mode 2 (Backwards) PWM1 Right --------- PWM2 Left
        if mode==0:
            if(rotationDirection == 1):
                PWM2 = avgSpeed + offset
                PWM1 = avgSpeed - offset  
            else:
                PWM2 = avgSpeed - offset
                PWM1 = avgSpeed + offset
        else:
            if(rotationDirection == -1):
                PWM2 = avgSpeed + offset
                PWM1 = avgSpeed - offset
            else:
                PWM2 = avgSpeed - offset
                PWM1 = avgSpeed + offset
    print("target_centre ",target_centre)
    print("robot_centre ",robot_centre)
    print("robot_heading ",robot_heading)
    print("target_vector ",target_vector)
    print("target_angle ",target_angle)
    print("heading_error ",heading_error)
    print("absHeadingError ",absHeadingError)
    print("PWM1 ",PWM1,"PWM2 ",PWM2,"mode ",mode)
    message = str(PWM1) + ',' + str(PWM2) + ',' + str(mode)
    ###
    data = message.encode()
    sock.sendto(data, (UDP_IP, UDP_PORT))
#     bytesAddressPair = sock.recvfrom(1024)
#     message = bytesAddressPair[0].decode()
#     address = bytesAddressPair[1]
    print(message)
    ###
    
    if cv2.waitKey(1) == 13: #13 Enter Key
        break
    #send signal to arduino
#      if list_ids[0] == 5:
#         data = str(0).encode()
#     elif avg_intensity > 80:
#         data = str(1).encode()
#     else:
#         data = str(-1).encode()
    
#     sock.sendto(data, (UDP_IP, UDP_PORT))
    
#     bytesAddressPair = sock.recvfrom(1024)

#     message = bytesAddressPair[0].decode()
#     address = bytesAddressPair[1]
#     print(message)
     #,end="\r"


cap.release()                                             # For releasing cap and out. 
cv2.destroyAllWindows()


on
target_centre  [580.5 277. ]
robot_centre  [435.5 362.5]
robot_heading  -45.0
target_vector  [145.  -85.5]
target_angle  120.52594707428098
heading_error  165.52594707428096
absHeadingError  14.474052925719036
PWM1  155.80411405142883 PWM2  155.80411405142883 mode  -1
155.80411405142883,155.80411405142883,-1
target_centre  [580.5 277. ]
robot_centre  [435.25 362.25]
robot_heading  -43.49256424122503
target_vector  [145.25 -85.25]
target_angle  120.40948675012154
heading_error  163.90205099134658
absHeadingError  16.097949008653416
PWM1  155.89433050048075 PWM2  155.89433050048075 mode  -1
155.89433050048075,155.89433050048075,-1
target_centre  [580.5 277. ]
robot_centre  [435.75 362.75]
robot_heading  -43.49256424122503
target_vector  [144.75 -85.75]
target_angle  120.64252973739549
heading_error  164.1350939786205
absHeadingError  15.864906021379511
PWM1  155.8813836678544 PWM2  155.8813836678544 mode  -1
155.8813836678544,155.8813836678544,-1
target_centre  [580.5 277. ]
robot_cen

target_centre  [580.5 277. ]
robot_centre  [435.75 354.25]
robot_heading  -52.49585763972986
target_vector  [144.75 -77.25]
target_angle  118.08789316820902
heading_error  170.58375080793888
absHeadingError  9.416249192061116
PWM1  155.5231249551145 PWM2  155.5231249551145 mode  -1
155.5231249551145,155.5231249551145,-1
target_centre  [580.5 277. ]
robot_centre  [435.75 353.75]
robot_heading  -52.49585763972986
target_vector  [144.75 -76.75]
target_angle  117.93363198403112
heading_error  170.42948962376101
absHeadingError  9.570510376238985
PWM1  155.53169502090216 PWM2  155.53169502090216 mode  -1
155.53169502090216,155.53169502090216,-1
target_centre  [580.5 277. ]
robot_centre  [436.   353.25]
robot_heading  -54.833563964207116
target_vector  [144.5  -76.25]
target_angle  117.81978891671913
heading_error  172.65335288092626
absHeadingError  7.346647119073737
PWM1  155.40814706217077 PWM2  155.40814706217077 mode  -1
155.40814706217077,155.40814706217077,-1
target_centre  [580.5 277

173.82619518118304,173.82619518118304,2
366.46691528704196
rotationDirection  -1
target_centre  [580.5 277. ]
robot_centre  [469.5 328. ]
robot_heading  -64.35899417569472
target_vector  [111. -51.]
target_angle  114.67686317033706
heading_error  179.03585734603178
absHeadingError  0.9641426539682243
PWM1  173.3233457643521 PWM2  173.3233457643521 mode  2
173.3233457643521,173.3233457643521,2
354.7341434088351
rotationDirection  -1
target_centre  [580.5 277. ]
robot_centre  [473.   326.25]
robot_heading  -64.39781244855823
target_vector  [107.5  -49.25]
target_angle  114.61438791757658
heading_error  179.01220036613483
absHeadingError  0.9877996338651656
PWM1  172.73670717044175 PWM2  172.73670717044175 mode  2
172.73670717044175,172.73670717044175,2
343.99582119554884
rotationDirection  -1
target_centre  [580.5 277. ]
robot_centre  [476.25 324.75]
robot_heading  -64.85521436932105
target_vector  [104.25 -47.75]
target_angle  114.60936977955512
heading_error  179.46458414887616
absHead

280.4097448377998
rotationDirection  1
target_centre  [615.5 267.5]
robot_centre  [524.25 287.75]
robot_heading  -82.47617956136138
target_vector  [ 91.25 -20.25]
target_angle  102.51218578344636
heading_error  -175.01163465519227
absHeadingError  4.9883653448077325
PWM1  170.01816031085153 PWM2  168.02281417292843 mode  2
170.01816031085153,168.02281417292843,2
target_centre  [616. 267.]
robot_centre  [524.  286.5]
robot_heading  -83.6598082540901
target_vector  [ 92.  -19.5]
target_angle  101.96710237658496
heading_error  -174.37308936932493
absHeadingError  5.62691063067507
PWM1  155.31260614614862 PWM2  155.31260614614862 mode  1
155.31260614614862,155.31260614614862,1
target_centre  [616.  267.5]
robot_centre  [525.   285.25]
robot_heading  -86.82016988013577
target_vector  [ 91.   -17.75]
target_angle  101.03724176956764
heading_error  -172.1425883502966
absHeadingError  7.857411649703408
PWM1  155.43652286942796 PWM2  155.43652286942796 mode  1
155.43652286942796,155.43652286942

303.89070815015054
rotationDirection  1
target_centre  [475.5 181.5]
robot_centre  [546.5  253.75]
robot_heading  -138.71528910542878
target_vector  [-71.   -72.25]
target_angle  -135.49995048300804
heading_error  3.215338622420745
absHeadingError  3.215338622420745
PWM1  169.55146768302336 PWM2  170.83760313199167 mode  0
169.55146768302336,170.83760313199167,0
target_centre  [475.5 181.5]
robot_centre  [546.5  253.25]
robot_heading  -142.02839623894963
target_vector  [-71.   -71.75]
target_angle  -135.3010258440964
heading_error  6.727370394853239
absHeadingError  6.727370394853239
PWM1  155.37374279971408 PWM2  155.37374279971408 mode  1
155.37374279971408,155.37374279971408,1
target_centre  [475.5 181.5]
robot_centre  [546.5  253.25]
robot_heading  -142.02839623894963
target_vector  [-71.   -71.75]
target_angle  -135.3010258440964
heading_error  6.727370394853239
absHeadingError  6.727370394853239
PWM1  155.37374279971408 PWM2  155.37374279971408 mode  1
155.37374279971408,155.3737

...

target_centre  [580. 175.]
robot_centre  [230.5 234. ]
robot_heading  90.0
target_vector  [349.5 -59. ]
target_angle  99.58190970032851
heading_error  9.58190970032851
absHeadingError  9.58190970032851
PWM1  155.53232831668493 PWM2  155.53232831668493 mode  1
155.53232831668493,155.53232831668493,1
target_centre  [573.5 177. ]
robot_centre  [230.5 234. ]
robot_heading  90.0
target_vector  [343. -57.]
target_angle  99.43523200696484
heading_error  9.435232006964839
absHeadingError  9.435232006964839
PWM1  155.52417955594248 PWM2  155.52417955594248 mode  1
155.52417955594248,155.52417955594248,1
target_centre  [567.5 179. ]
robot_centre  [230.5 234. ]
robot_heading  90.0
target_vector  [337. -55.]
target_angle  99.26922209340117
heading_error  9.269222093401169
absHeadingError  9.269222093401169
PWM1  155.51495678296672 PWM2  155.51495678296672 mode  1
155.51495678296672,155.51495678296672,1
target_centre  [562.5 181. ]
robot_centre  [230.5 234. ]
robot_heading  90.0
target_vector  [332

894.0006291944094
rotationDirection  1
target_centre  [543.5 194. ]
robot_centre  [247.25 226.25]
robot_heading  95.38931175997341
target_vector  [296.25 -32.25]
target_angle  96.2127972265226
heading_error  0.8234854665491866
absHeadingError  0.8234854665491866
PWM1  199.70003145972046 PWM2  199.70003145972046 mode  0
199.70003145972046,199.70003145972046,0
898.9700356519121
rotationDirection  1
target_centre  [545.5 193.5]
robot_centre  [247.75 227.25]
robot_heading  93.23970029610213
target_vector  [297.75 -33.75]
target_angle  96.46688204140008
heading_error  3.227181745297969
absHeadingError  3.227181745297969
PWM1  199.303065433536 PWM2  200.5939381316552 mode  0
199.303065433536,200.5939381316552,0
target_centre  [553.5 191.5]
robot_centre  [256.75 235.75]
robot_heading  80.36246188706906
target_vector  [296.75 -44.25]
target_angle  98.48119158870779
heading_error  18.118729701638728
absHeadingError  18.118729701638728
PWM1  156.0065960945355 PWM2  156.0065960945355 mode  1
156.

765.5652323610313
rotationDirection  -1
target_centre  [584. 182.]
robot_centre  [332.25 223.75]
robot_heading  99.63753811293094
target_vector  [251.75 -41.75]
target_angle  99.41618292059768
heading_error  -0.22135519233324885
absHeadingError  0.22135519233324885
PWM1  193.27826161805157 PWM2  193.27826161805157 mode  0
193.27826161805157,193.27826161805157,0
764.5829745684898
rotationDirection  1
target_centre  [584.5 182. ]
robot_centre  [333.25 224.75]
robot_heading  97.52382043863864
target_vector  [251.25 -42.75]
target_angle  99.65635652023974
heading_error  2.1325360816011028
absHeadingError  2.1325360816011028
PWM1  192.80264151210426 PWM2  193.65565594474472 mode  0
192.80264151210426,193.65565594474472,0
target_centre  [584.5 182. ]
robot_centre  [334.5 226. ]
robot_heading  94.23639479905884
target_vector  [250. -44.]
target_angle  99.98182928773316
heading_error  5.745434488674334
absHeadingError  5.745434488674334
PWM1  155.31919080492636 PWM2  155.31919080492636 mode  1

524.3782032083332
rotationDirection  -1
target_centre  [554.5 176.5]
robot_centre  [384. 215.]
robot_heading  104.53445508054013
target_vector  [170.5 -38.5]
target_angle  102.72435568542238
heading_error  -1.8100993951177315
absHeadingError  1.8100993951177315
PWM1  181.21891016041667 PWM2  181.21891016041667 mode  0
181.21891016041667,181.21891016041667,0
513.3118569641656
rotationDirection  -1
target_centre  [551.5 176.5]
robot_centre  [384.5  213.75]
robot_heading  105.52411099675426
target_vector  [167.   -37.25]
target_angle  102.57420847580867
heading_error  -2.949902520945585
absHeadingError  2.949902520945585
PWM1  181.25557335239742 PWM2  180.07561234401916 mode  0
181.25557335239742,180.07561234401916,0
500.6231741539738
rotationDirection  -1
target_centre  [549.5 176.5]
robot_centre  [386.5  212.25]
robot_heading  106.09081634885217
target_vector  [163.   -35.75]
target_angle  102.37053247773738
heading_error  -3.7202838711147876
absHeadingError  3.7202838711147876
PWM1  18

283.9975792150349
rotationDirection  -1
target_centre  [549. 174.]
robot_centre  [456.75 195.25]
robot_heading  105.25511870305779
target_vector  [ 92.25 -21.25]
target_angle  102.97193437840903
heading_error  -2.2831843246487438
absHeadingError  2.2831843246487438
PWM1  169.65651582568148 PWM2  168.74324209582198 mode  0
169.65651582568148,168.74324209582198,0
280.17494534665303
rotationDirection  -1
target_centre  [549. 174.]
robot_centre  [458. 195.]
robot_heading  104.53445508054013
target_vector  [ 91. -21.]
target_angle  102.9946167919165
heading_error  -1.5398382886236277
absHeadingError  1.5398382886236277
PWM1  169.00874726733264 PWM2  169.00874726733264 mode  0
169.00874726733264,169.00874726733264,0
273.4950867931634
rotationDirection  -1
target_centre  [549. 174.]
robot_centre  [460.   193.75]
robot_heading  105.52411099675426
target_vector  [ 89.   -19.75]
target_angle  102.51176553847235
heading_error  -3.0123454582819136
absHeadingError  3.0123454582819136
PWM1  169.2772